# Mujin Document Engineer Coding Challenge
## Samir Wadhwania, March 1, 2021

I want to thank Natsumi, Yuto, and especially Ziyan for giving me the opportunity to interview with Mujin and the help along the way. This coding challenge proved to be more difficult than expected (infuriatingly so), but I have done my best to recover as much as possible and will document both my code and my process below. Thank you.

### Challenges in the beginning

I installed Docker on my workstation and pulled the *debian:stretch* image from Docker Hub. From there, I cloned the openRAVE repository and checked out the *production* branch. As the image was a clean install of *debian*, I expected to install several packages along the way, but did not expect to get completely stuck during the compiling process.

Small challenges here and there weren't too much of an issue: installing most from aptitude, some from source. However, apparently I erred (used the wrong versions, which seemed to be the common mistake at most turns) the most with rapidjson and fparser. Thank you to Ziyan for helping me identify the correct versions, which quickly resolved the issues.

There still seemed to be an issue actually compiling due to the *boost* libraries. I dug as deep as I could to see if it could be linked to another library or something unexpected, but after a while decided it would be best to focus on the rest of the challenge.

### Small cheat(!)

After talking to Ziyan, I decided to build out the HTTP server with the last 12 hours remaining in the challenge. He suggested using dummy JSON files to emulate the task. I realized partway through that a docker image with openRAVE installed almost certainly exists on DockerHub. I actually found what I believe to be [Ziyan's own container](https://hub.docker.com/r/ziyan/openrave) with openRAVE installed. I pulled that image and worked onward from there.

## README

I would like to immediately credit the following people / sites where I learned and drew inspiration from:

- [Geeks for Geeks: POST Requests Using Python](https://www.geeksforgeeks.org/get-post-requests-using-python/)
- [Miel Donkers: Simple HTTPServer (albeit in Python3)](https://gist.github.com/mdonkers/63e115cc0c79b4f6b8b3a6b797e485c7)
- [W3 Schools: Python Requests](https://www.w3schools.com/python/ref_requests_post.asp)
- [And, of course, the openRAVE examples page / documentation](http://openrave.org/docs/latest_stable/openravepy/openravepy_int/)

### Structure

The term *collection* was ambiguous to me, so I interpreted it as the *set of robots present in a given simulated environment*. If this was not the intended meaning, I apologize. As a result, there is a server-side script called **rave_server.py** that initializes an openRAVE environment and subsequently spins up a server to handle HTTP requests. Adding robots to the collection means adding them to the simulation, and modifying, removing, and listing robots are all interpreted similarly.

The server script accepts the following commands:

### GET Request
- **List Robots**: The default request (a GET request) returns a status code of 200 as well as the following information:
 - Success: 1 for success and 0 for failure
 - Error message: Empty for success, specific for failure
 - Robots: A list of robots in the collection by name

### POST Request
The only parameter common to all post requests is the *type* parameter. It is used to discriminate what the client would like to do, and looks for the appropriate accompanying information.
- **add**: Adds a robot to the simulation. Requires:
 - xml: a JSONified *.dae* file containing robot information
- **remove**: Removes a robot from the simulation. Requires:
 - robot: The robot name in the environment
- **set**: Modifies a robot already in the simulation. Requires:
 - robot: The robot name in the environment
 - set: A *dict* of attributes to change and their corresponding values. The following are acceptable attributes (note: server expects perfectly formatted/sized information):
   - DOFAccelerationLimits
   - DOFLimits
   - DOFTorqueLimits
   - DOFValues
   - DOFVelocityLimits
   - DOFWeights
   - Name
- **download**: I wanted to implement the *add* functionality in reverse: take the robot XML data from the openRAVE environment, JSONify it, and return it to the client to convert back to and save as a *.dae* file. However, within the openRAVE environment docs, I could not (within the time) gather how to save the information of a single Robot/KinBody/Interface. Perhaps my interpretation of "collection" makes this difficult for me.

The server makes the requested changes, and sends back the same information as above.

### Errors
If the *type* parameter is not one of the listed values, it returns an error code (418) and flags a failed request as well as a description of the error. 

All other errors are assumed to be syntax errors (misformated request) and returned with code 400.

## Client Side Scripts

Below are scripts that can be run to test out the functionality of the HTTP server. After running the docker image with `-p 8888:8888`, I run the following in the container:

`python rave_server.py -l "0.0.0.0" -p 8888 -e "data/lab1.env.xml"`

Then, I use the scripts below:

In [3]:
import requests
import simplejson

url = 'http://localhost:8888/'

### List robots

In [10]:
# Make a GET request
r = requests.get(url)

In [11]:
# Examine the response
print("Status Code: {}".format(r.status_code))
print("Response: {}".format(r.json()))

Status Code: 200
Response: {'Error Message': '', 'Success': 1, 'Robots': ['BarrettWAM']}


We can see that the only Robot in the environment is *BarrettWAM*

### Add a robot 

Let's add the *kuka-youbot* robot.

In [41]:
# JSONify the .dae file
with open('robots/kuka-youbot.dae', 'r') as file:
    data = {
        'type': 'add', 
        'xml': simplejson.dumps(file.read())
    }
    
r = requests.post(url, json=data)

In [42]:
# Examine the response
print("Status Code: {}".format(r.status_code))
print("Response: {}".format(r.json()))

Status Code: 200
Response: {'Error Message': '', 'Success': 1, 'Robots': ['BarrettWAM', 'youbot']}


### Modify a robot

We can see that the *kuka-youbot* has been added. Let's change that name to something more fun, and change some limits.

In [53]:
data = {
    'type': 'set', 
    'robot': 'youbot', 
    'set': {'Name': 'Turing', 
            'DOFAccelerationLimits': [1, 1, 1, 1, 1, 1, 1]
           }
    }

r = requests.post(url, json=data)

In [54]:
# Examine the response
print("Status Code: {}".format(r.status_code))
print("Response: {}".format(r.json()))

Status Code: 200
Response: {'Error Message': '', 'Success': 1, 'Robots': ['BarrettWAM', 'Turing']}


### Remove a robot

The robot has been changed, and internally, the DOFAccelerationLimits have been set as well. Let's get rid of our original *BarrettWAM* robot.

In [55]:
data = {
    'type': 'remove', 
    'robot': 'BarrettWAM', 
    }

r = requests.post(url, json=data)

In [56]:
# Examine the response
print("Status Code: {}".format(r.status_code))
print("Response: {}".format(r.json()))

Status Code: 200
Response: {'Error Message': '', 'Success': 1, 'Robots': ['Turing']}


Our original is gone, and we are left with just our own!

## Frameworks / Tool Choices

I opted for a SimpleHTTP server because it would have been the most straightforward. However, I wouldn't want to use this for production because there are very few security checks involved, but it sufficed for this coding challenge.

As you may have noticed, I enjoy using iPython / Jupyter notebooks. I ran a jupyter server in the Docker container so I could work in my host computer's browser, and I also ran a jupyter server on my host computer so I could tinker with sending requests and received responses.

Python is the most straightforward for me, and I did my best to adhere to PEP8 standards.

## Improvements

This is nowhere near how I would want something professional to look. Just the notes that I made during the process:

- Implementing a wrapper for handling different types of information (string, nparray, etc).
- Implementing a POST request to get the current attributes of a specific robot (started to do this, realized I needed the wrappers, ran out of time)
- Create a python executable for the client side so you don't have to run scripts out of a Jupyter notebook
- One step further, build a GUI
- I feel like my route of a global environment variable is really ugly. Definitely a more elegant way of implementing that.

And I cannot get this far without talking about my compilation issues. It shouldn't have been as difficult as it was, and most of was probably/definitely my fault. That being said -- there's no documentation about installing openRAVE on the repository past 2016! I manually searched for several packages that were required, and had to dig to find the specific versions of some. I hope that this deliverable was still able to accomplish the original goal.